In [ ]:
from google.colab import files
uploaded = files.upload()


Saving colored_images.zip to colored_images.zip


In [ ]:
import os
os.listdir()


['.config', 'colored_images.zip', 'sample_data']

In [ ]:
import zipfile
import os

zip_path = "colored_images.zip"  # Update the filename if needed
extract_path = "colored_images"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete. Files are stored in:", extract_path)


Extraction complete. Files are stored in: colored_images


In [ ]:
import tensorflow as tf
import os

# Parameter
data_dir = "/content/colored_images"
img_size = (224, 224)
batch_size = 32
seed = 123

# Membuat dataset training (70%) dengan validation_split=0.3 (subset="training")
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.3,
    subset="training",
    seed=seed,
    image_size=img_size,
    batch_size=batch_size
)

# Membuat dataset sementara (30%) dengan subset="validation"
temp_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.3,
    subset="validation",
    seed=seed,
    image_size=img_size,
    batch_size=batch_size
)

# Memisahkan dataset sementara menjadi validation (15%) dan test (15%)
temp_ds_card = tf.data.experimental.cardinality(temp_ds).numpy()
val_ds = temp_ds.take(temp_ds_card // 2)
test_ds = temp_ds.skip(temp_ds_card // 2)

# Optimasi pipeline dengan prefetching
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds   = val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds  = test_ds.prefetch(buffer_size=AUTOTUNE)

Found 3662 files belonging to 5 classes.
Using 2564 files for training.
Found 3662 files belonging to 5 classes.
Using 1098 files for validation.


In [ ]:
from tensorflow.keras import layers, models

def create_resnet50_model():
    base_model = tf.keras.applications.ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=img_size + (3,)
    )
    base_model.trainable = False  # freeze base model

    inputs = tf.keras.Input(shape=img_size + (3,))
    x = tf.keras.applications.resnet.preprocess_input(inputs)
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(5, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
def create_efficientnet_model():
    base_model = tf.keras.applications.EfficientNetB0(
        weights='imagenet',
        include_top=False,
        input_shape=img_size + (3,)
    )
    base_model.trainable = False

    inputs = tf.keras.Input(shape=img_size + (3,))
    x = tf.keras.applications.efficientnet.preprocess_input(inputs)
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(5, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
def create_mobilenet_model():
    base_model = tf.keras.applications.MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=img_size + (3,)
    )
    base_model.trainable = False

    inputs = tf.keras.Input(shape=img_size + (3,))
    x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(5, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
def create_inceptionv3_model():
    base_model = tf.keras.applications.InceptionV3(
        weights='imagenet',
        include_top=False,
        input_shape=img_size + (3,)
    )
    base_model.trainable = False

    inputs = tf.keras.Input(shape=img_size + (3,))
    x = tf.keras.applications.inception_v3.preprocess_input(inputs)
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(5, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
from tensorflow.keras import callbacks

def train_model(model, model_name, epochs=10):
    es = callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    cp = callbacks.ModelCheckpoint(f'best_{model_name}.keras', monitor='val_loss', save_best_only=True)

    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs,
        callbacks=[es, cp]
    )

    test_loss, test_accuracy = model.evaluate(test_ds)
    return {'test_loss': test_loss, 'test_accuracy': test_accuracy, 'history': history}

In [ ]:
results = {}

# ResNet50
resnet50_model = create_resnet50_model()
results['ResNet50'] = train_model(resnet50_model, 'ResNet50', epochs=10)

# EfficientNetB0
efficientnet_model = create_efficientnet_model()
results['EfficientNetB0'] = train_model(efficientnet_model, 'EfficientNetB0', epochs=10)

# MobileNetV2
mobilenet_model = create_mobilenet_model()
results['MobileNetV2'] = train_model(mobilenet_model, 'MobileNetV2', epochs=10)

# InceptionV3
inceptionv3_model = create_inceptionv3_model()
results['InceptionV3'] = train_model(inceptionv3_model, 'InceptionV3', epochs=10)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Epoch 1/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 607s 7s/step - accuracy: 0.6276 - loss: 1.0742 - val_accuracy: 0.7390 - val_loss: 0.6808
Epoch 2/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 653s 8s/step - accuracy: 0.7852 - loss: 0.6091 - val_accuracy: 0.7812 - val_loss: 0.6443
Epoch 3/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 592s 7s/step - accuracy: 0.7999 - loss: 0.5481 - val_accuracy: 0.7960 - val_loss: 0.5850
Epoch 4/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 622s 7s/step - accuracy: 0.8040 - loss: 0.5175 - val_accuracy: 0.7996 - val_loss: 0.5752
Epoch 5/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 591s 7s/step - accuracy: 0.8096 - loss: 0.4933 - val_accuracy: 0.7757 - val_loss: 0.6456
Epoch 6/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 593s 7s/step - accuracy: 0.8264 - loss: 0.4504 - val_accuracy: 0.8033 - val_loss: 0.5663
Epoch 7/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 623s 7s/step - accuracy: 0.8473 - loss: 0.4296 - val_accuracy: 0.8088 - val_loss: 0.5453
Epoch 8/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 657s 8s/step - accurac

In [ ]:
import pandas as pd

# Membuat DataFrame hasil komparasi
results_df = pd.DataFrame(results).T
results_df = results_df[['test_accuracy', 'test_loss']]
print(results_df)

               test_accuracy test_loss
ResNet50            0.810469  0.539136
EfficientNetB0      0.792419  0.560646
MobileNetV2         0.788809  0.551815
InceptionV3         0.758123  0.656559
